<h1 style="text-align: center; vertical-align: middle;">Numerical Methods in Accelerator Physics</h1>
<h2 style="text-align: center; vertical-align: middle;">Python examples <span style="color:darkred">and tasks</span> -- Week 6</h2>

<h2>Run this first!</h2>

Imports and modules:

In [ ]:
from config6 import (np, plt, plot_rfwave, tqdm, trange, 
                     beta, gamma, Machine, track_one_turn, 
                     charge, mass, emittance, hamiltonian,
                     plot_hamiltonian, plot_hamiltonian_ax, 
                     plot_rf_overview, plot_dist, plot_mp)
from scipy.constants import m_p, e, c
import matplotlib.animation as animation 
%matplotlib inline

If the progress bar by `tqdm` (`trange`) later in this document does not work, run this:

In [ ]:
#!jupyter nbextension enable --py widgetsnbextension

<h2>Simulation of Full CERN PS Acceleration Ramp</h2>
<h3>CERN PS Machine parameters</h3>

The CERN Proton Synchrotron
- has a circumference of 2π·100m
- takes protons from the PS Booster at a kinetic energy of 2GeV corresponding to a $\gamma=3.13$
- injects with 50kV of rf voltage, up to 200kV for ramp
- runs at harmonic $h=7$
- has a momentum compaction factor of $\alpha_c=0.027$
- typical acceleration rate of (up to) $\dot{B}=2$ T/s, the bending radius is $\rho=70.08$ m

We start by instantiating the PS, `Machine(...)`:

(<i>hint: hit both `Shift+Tab` keys inside the parentheses `()` below to get info about the possible arguments to `Machine` and the initial values</i>)

In [ ]:
m = Machine()

assert m.phi_s > 0, "machine is not accelerating...?"

You can check the rf systems setup by the following plot command from the previous lecture:

In [ ]:
plot_rf_overview(m);

We initialise a Gaussian bunch distribution with very small rms bunch length $\sigma_z=1$ m (so that the small-amplitude approximation holds):

In [ ]:
sigma_z=1

The "matched" rms momentum spread $\sigma_{\Delta p}$ (remember, $\sigma_{\Delta p}$ and $\sigma_z$ are linked via equal Hamiltonian values $\mathcal{H}_0$, the equilibrium condition):

In [ ]:
sigma_deltap = np.sqrt(
    2 * m.p0() / -m.eta(0) * 
    charge * m.voltage * np.pi * m.harmonic / (beta(gamma(m.p0())) * c * m.circumference**2)
) * sigma_z
sigma_deltap

<h3>Generating Macro-particles via Box-Muller</h3>

Limit by machine precision: the smallest number at FP64 is $\varepsilon\approx2^{-53}$, therefore one can only generate pseudo-random numbers from the Gaussian distribution up to an amplitude of $x$ where $\exp(-x^2/2)=2^{-53}$, i.e.

In [ ]:
np.sqrt(2*-np.log(2**-53))

$\implies$ given $\sigma_z=1\,$</i>m<i>, no particles can be generated outside of $z=8.6\,$m and the equivalent Hamiltonian contour in phase space)

In [ ]:
N = 1000 # Number of macro-particles

In [ ]:
np.random.seed(12345) # set seed to get always the same "random" distribution

z = np.random.normal(loc=0, scale=sigma_z, size=N) # generate N particle positions with sigma_z around z=0
deltap = np.random.normal(loc=0, scale=sigma_deltap, size=N) # generate N particle momenta with sigma_deltap around deltap=0

In [ ]:
plot_hamiltonian(m)
plt.scatter(z, deltap / m.p0(), marker='.', s=1)

<h3> Compute duration of ramp</h3>

$(\Delta\gamma)_\mathrm{turn} = \cfrac{\Delta E_\mathrm{tot}}{m_0c^2} = \cfrac{qV\sin(\varphi_s)}{m_0c^2}$

such that accelerating from $\gamma_\mathrm{ref}=3.1$ to $\gamma_\mathrm{ref}=27.7$ takes as many turns as

$n_\mathrm{turns}=\cfrac{27.7-3.1}{(\Delta\gamma)_\mathrm{turn}}$

In [ ]:
dgamma_per_turn = charge * m.voltage * np.sin(m.phi_s) / (mass * c**2)
n_turns = int(np.ceil((27.7-3.13) / dgamma_per_turn))
n_turns

Record longitudinal emittance during tracking:

In [ ]:
epsn_z = np.zeros(n_turns, dtype=np.float64)
epsn_z[0] = emittance(z, deltap)

Let's go tracking!

In [ ]:
for i_turn in trange(1, n_turns):
    z, deltap = track_one_turn(z, deltap, m)
    epsn_z[i_turn] = emittance(z, deltap)

<h3>How can we check the results?</h3>

At first, have we reached the extraction energy of $\gamma=27.7$?

In [ ]:
m.gamma_ref

Yes, we did.

Let's check the rms emittance next. We have started with a matched particle distribution. Thus, we expect to have a constant emittance along the acceleration ramp. How does it look like in our results?

In [ ]:
plt.plot(np.arange(n_turns) / 100000, epsn_z / e)
plt.xlabel('Turns [100k]')
plt.ylabel('$\epsilon_z$ [eV.s]');

<b>$\implies$ Something went wrong, since emittance has increased significantly after a few thousand turns!</b>

In the last exercise we have already calculated that the acceleration ramp crosses transition energy. With the momentum compaction factor $\alpha_c$ and the energy gain per turn $\left(\Delta\gamma\right)_\textrm{turn}$, we can determine the number of turns until reaching transition energy:

In [ ]:
gamma_t = np.sqrt(1/m.alpha_c)
int(np.ceil((gamma_t-3.13) / dgamma_per_turn))

This fits very well to the start of the observed emittance increase.

Furthermore, you can use the following phase-space plots as diagnostics. For this, you can stop the tracking after a certain turn to investigate, e.g. by changing the value of `n_turns` inside the `trange` counter in the `for` loop.

In [ ]:
plot_hamiltonian(m);
plt.scatter(z % 600, deltap / m.p0(), marker='.', s=1)

However, we can also animate the phase space evolution using the python package celluloid. (So far, it was not included in the python environment and we have to install it first.)

In [ ]:
import sys
!{sys.executable} -m pip install celluloid

In [ ]:
from celluloid import Camera

np.random.seed(12345)

z = np.random.normal(loc=0, scale=sigma_z, size=N)
deltap = np.random.normal(loc=0, scale=sigma_deltap, size=N)

m = Machine()

fig, ax = plt.subplots(2)
camera = Camera(fig)

ax[0].set_xlim([-50,50])
ax[0].set_ylim([-0.01,0.01])

ax[0].set_xlabel('z [m]')
ax[1].set_xlabel('z [m]')
ax[0].set_ylabel(r'$\delta$')
ax[1].set_ylabel(r'$\delta$')

for j_turn in trange(0, int(n_turns/1000)):
    for i_turn in range(1,999):
        z, deltap = track_one_turn(z, deltap, m)
    plot_hamiltonian_ax(m,ax[0])
    ax[0].scatter(z,deltap/m.p0(), marker='.', s=1)
    ax[1].scatter(z,deltap/m.p0(), marker='.', s=1)
    ax[0].text(0.4, 1.05, f'turn {j_turn*1000+999}', transform=ax[0].transAxes)
    plt.tight_layout()
    camera.snap()

ani = camera.animate(blit=False)

from IPython.display import HTML
HTML(ani.to_jshtml())

<h3>Crossing transition</h3>

In order to stabilize the beam and reduce the emittance growth, we need to change the synchronous phase at transition. For this, we add a phase jump in the tracking loop:

In [ ]:
np.random.seed(12345) # set seed to get always the same "random" distribution

z = np.random.normal(loc=0, scale=sigma_z, size=N) # generate N particle positions with sigma_z around z=0
deltap = np.random.normal(loc=0, scale=sigma_deltap, size=N) # generate N particle momenta with sigma_deltap around deltap=0

m = Machine()

In [ ]:
phi_s_1 = np.pi - m.phi_s # synchronous phase after transition calculated from synchronous phase before transition

for i_turn in trange(1, n_turns):
    z, deltap = track_one_turn(z, deltap, m)
    
    epsn_z[i_turn] = emittance(z, deltap)
    
    condition = m.gamma_ref > np.sqrt(1/m.alpha_c) # condition to change synchronous phase: gamma above gamma_transition
    
    if condition:
        m.phi_s = phi_s_1

In [ ]:
plt.plot(np.arange(n_turns) / 100000, epsn_z / e)
plt.xlabel('Turns [100k]')
plt.ylabel('$\epsilon_z$ [eV.s]');

$\implies$ Emittance conserved up to about 2% if phase jump at transition is included. An additional jump in transition energy can be introduced to further reduce the emittance growth.

We can also animate the phase space evolution again:

In [ ]:
from celluloid import Camera

np.random.seed(12345)

z = np.random.normal(loc=0, scale=sigma_z, size=N)
deltap = np.random.normal(loc=0, scale=sigma_deltap, size=N)

m = Machine()

fig, ax = plt.subplots()
camera = Camera(fig)

ax.set_xlim([-50,50])
ax.set_ylim([-0.01,0.01])

ax.set_xlabel('z [m]')
ax.set_ylabel(r'$\delta$')

phi_s_1 = np.pi - m.phi_s

for j_turn in trange(0, int(n_turns/1000)):
    for i_turn in range(1,999):
        z, deltap = track_one_turn(z, deltap, m)

        condition = m.gamma_ref > np.sqrt(1/m.alpha_c) # condition to change synchronous phase: gamma above gamma_transition
    
        if condition:
            m.phi_s = phi_s_1
    plot_hamiltonian_ax(m,ax)
    ax.scatter(z,deltap/m.p0(), marker='.', s=1)
    ax.text(0.4, 1.05, f'turn {j_turn*1000+999}', transform=ax.transAxes)
    plt.tight_layout()
    camera.snap()

ani = camera.animate(blit=False)

from IPython.display import HTML
HTML(ani.to_jshtml())

<span style="color:darkred">The emittance growth can be further reduced by introducing a jump in transition energy. You can try to implement this using the getter and setter functions `m.get_alpha_c()`, `m.set_alpha_c(alpha_c)` and `m.get_gamma_ref()`. A possible jump starts with an increase in $\gamma_t$ at the turn with $\gamma=\gamma_{t,0}-0.5$ and with the same gradient as the acceleration ramp up to $\gamma=\gamma_{t,0}$, followed by a quick change below the particles' energy and an increase with the same gradient up to $\gamma_{t,0}$.</span>

In [ ]:
...

<h2>PyHEADTAIL</h2>
<h3>RF Buckets in PyHEADTAIL</h3>

`PyHEADTAIL` provides a class to represent rf buckets (for plotting as well as for matching)

In [ ]:
from PyHEADTAIL.trackers.rf_bucket import RFBucket

We define a convenience function to provide `RFBucket` instance given a `Machine` instance:

In [ ]:
def get_pyht_rfbucket(machine):
    m = machine
    deltap_per_turn = charge * m.voltage / (beta(gamma(m.p0())) * c) * np.sin(m.phi_s)
    rfb = RFBucket(m.circumference, m.gamma_ref, mass, charge, [m.alpha_c], deltap_per_turn, [m.harmonic], [m.voltage], [np.pi + m.phi_s])
    # PyHEADTAIL has a different convention for the phase and is offset by pi compared to our lecture
    return rfb

<h3>Visualising the Distributions in the RF Bucket</h3>

In [ ]:
m = Machine(gamma_ref=3.13)

In [ ]:
rfb = get_pyht_rfbucket(m)

In [ ]:
from PyHEADTAIL.particles.rfbucket_matching import (ThermalDistribution, WaterbagDistribution, ParabolicDistribution)

In [ ]:
sigma_z = 8

Computing the (initial) guess for $\mathcal{H}_0$ based on the small-amplitude approximation:

In [ ]:
H0 = rfb.guess_H0(sigma_z, from_variable='sigma')
H0

In [ ]:
plot_dist(ThermalDistribution, rfb, H0=0.05*H0);

In [ ]:
plot_dist(WaterbagDistribution, rfb, H0=2*H0)

In [ ]:
plot_dist(ParabolicDistribution, rfb, H0=H0*2)

Matching algorithm implemented in the `RFBucketMatcher` class in `PyHEADTAIL`


In [ ]:
from PyHEADTAIL.particles.generators import RFBucketMatcher

In [ ]:
rfb_matcher = RFBucketMatcher(rfb, ThermalDistribution, sigma_z=sigma_z)
rfb_matcher.integrationmethod = 'cumtrapz'

Calling the `RFBucketMatcher.generate` method will 

(1.) iterate on $\mathcal{H}_0$ until the numerical integration of $\psi(\mathcal{H})$ for the bunch length converges to $\hat{\sigma}_z$, and then 

(2.) sample this $\psi(\mathcal{H})$ by <b>rejection sampling</b> (see previous lecture) to generate the macro-particle phase-space coordinates $(z,\delta)$

In [ ]:
z, delta, _, _ = rfb_matcher.generate(int(1e5))

In [ ]:
print("Converged value of H0: " + str(rfb_matcher.psi_object.H0))
print("Small-amplitude approximation: " + str(rfb.guess_H0(sigma_z, from_variable='sigma')))

$\implies$ For smaller target $\hat{\sigma}_z$ the values are closer together.

Let's have a look at the generated macro-particle distribution:

In [ ]:
plot_mp(z, delta, rfb);

Does the rms bunch length of the macro-particle distribution match the chosen target $\hat{\sigma}_z$?

In [ ]:
print("RMS bunch length: " + str(np.std(z)))
print("Target sigma_z: " + str(sigma_z))

<h2>Root-finding</h2>

<h3>Let's use scipy.optimize ...</h3>

In [ ]:
from scipy.optimize import brentq, newton

At what $x$ does $\cfrac{1}{\sqrt{x}+1}$ take the value `0.4`?

In [ ]:
def f(x):
    return 1 / (np.sqrt(x) + 1) - 0.4

Brent-Dekker algorithm with interval $x\in[0, 4]$:

In [ ]:
brentq(f, 0, 4)

<h3>... and Newton's Secant method</h3>

Secant method with initial values $x_0=0$, $x_1=10^{-4}$:

In [ ]:
newton(f,0)

<h3>How does the function look like?</h3>

In [ ]:
x = np.linspace(0, 4, 1000)

In [ ]:
plt.plot(x, f(x))
plt.axhline(0, c='k', lw=2)
plt.xlabel('$x$')
plt.ylabel('$f(x)$');

<h3>Implementing the Secant method</h3>

In [ ]:
def secant_method(f, x0, x1, iterations, rtol=2e-12):
    """Return the root calculated using the secant method."""
    for i in range(iterations):
        x2 = x1 - f(x1) * (x1 - x0) / float(f(x1) - f(x0))
        if np.abs(x2 - x1) / x1 < rtol:
            break
        x0, x1 = x1, x2
    return x2

In [ ]:
secant_method(f, 0, 1, 100)

(The `scipy` implementation of the secant method in `newton()` [computes](https://github.com/scipy/scipy/blob/v1.9.3/scipy/optimize/_zeros_py.py#L330) the second guess $x_1$ as $x_1=x_0\cdot 1.0001\pm 0.0001$.)

<h2>Numerical emittance growth</h2>

Start with the bi-Gaussian (simulation as previous lecture):

In [ ]:
m = Machine(phi_s=0)

In [ ]:
sigma_z = 13.5

In [ ]:
def generate_gaussian_in_rfbucket(N: int, sigma_z: float, machine: Machine, seed: int = 12345, margin: float = 0.05) -> tuple():
    '''Generate a bi-Gaussian distribution with N macro-particles,
    rms bunch length sigma_z and a matched sigma_deltap via the
    machine settings.
    '''
    np.random.seed(seed)
    m = machine
    
    sigma_deltap = np.sqrt(
        2 * m.p0() / -m.eta(0) * 
        charge * m.voltage * np.pi * m.harmonic / (beta(gamma(m.p0())) * c * m.circumference**2)
    ) * sigma_z

    z_ini = np.random.normal(loc=0, scale=sigma_z, size=N)
    deltap_ini = np.random.normal(loc=0, scale=sigma_deltap, size=N)
    
    H_safetymargin = margin * hamiltonian(0, 0, m)
    H_values = hamiltonian(z_ini, deltap_ini, m) - H_safetymargin

    while any(H_values >= 0):
        mask_bad = H_values >= 0
        N_bad = np.sum(mask_bad)
        print (N_bad)
        # re-initialise bad particles:
        z_ini[mask_bad] = np.random.normal(loc=0, scale=sigma_z, size=N_bad)
        deltap_ini[mask_bad] = np.random.normal(loc=0, scale=sigma_deltap, size=N_bad)
        # re-evaluate rejection condition
        H_values = hamiltonian(z_ini, deltap_ini, m) - H_safetymargin
    
    return z_ini, deltap_ini

In [ ]:
N = 10000
n_turns = 5000

In [ ]:
z_ini, deltap_ini = generate_gaussian_in_rfbucket(N, sigma_z, m)

In [ ]:
plot_mp(z_ini, deltap_ini / m.p0(), rfb=get_pyht_rfbucket(m), n_bins=20);

Track the bi-Gaussian distribution...

In [ ]:
z = np.zeros((n_turns, N), dtype=np.float64)
deltap = np.zeros_like(z)

z[0] = z_ini
deltap[0] = deltap_ini

In [ ]:
for i_turn in trange(1, n_turns):
    z[i_turn], deltap[i_turn] = track_one_turn(z[i_turn - 1], deltap[i_turn - 1], m)

The rms emittance evolution:

In [ ]:
epsn_z = np.array([emittance(z_i, deltap_i) for z_i, deltap_i in zip(z, deltap)])

ylim_m = np.median(4 * np.pi * epsn_z / e)
ylim_d = 1.1 * np.max(np.abs(ylim_m - 4 * np.pi * epsn_z / e))

In [ ]:
plt.plot(4 * np.pi * epsn_z / e)

plt.ylim(ylim_m - ylim_d, ylim_m + ylim_d)

plt.xlabel('Turns')
plt.ylabel('$4\pi\epsilon_z$ [eV.s]');

$\leadsto$ the Gaussian particle distribution is <b>not exactly</b> in equilibrium for sufficiently large rms values in the nonlinear potential, the particles <b>filament</b> and the rms emittance grows (a little)! 

$\implies$ compare to using full nonlinear Hamiltonian to construct PDF $\psi(\mathcal{H})\propto\exp\left(\cfrac{\mathcal{H}}{\mathcal{H}_0}\right)$

In [ ]:
rfb = get_pyht_rfbucket(m)

rfb_matcher = RFBucketMatcher(rfb, ThermalDistribution, sigma_z=sigma_z)
rfb_matcher.integrationmethod = 'cumtrapz'

In [ ]:
z_ini, delta_ini, _, _ = rfb_matcher.generate(N)

In [ ]:
deltap_ini = delta_ini * m.p0()

In [ ]:
plot_mp(z_ini, delta_ini, rfb, n_bins=20);

Track the matched thermal distribution...

In [ ]:
z = np.zeros((n_turns, N), dtype=np.float64)
deltap = np.zeros_like(z)

z[0] = z_ini
deltap[0] = deltap_ini

In [ ]:
for i_turn in trange(1, n_turns):
    z[i_turn], deltap[i_turn] = track_one_turn(z[i_turn - 1], deltap[i_turn - 1], m)

The rms emittance evolution:

The rms emittance evolution:

In [ ]:
epsn_z = np.array([emittance(z_i, deltap_i) for z_i, deltap_i in zip(z, deltap)])

ylim_m = np.median(4 * np.pi * epsn_z / e)

In [ ]:
plt.plot(4 * np.pi * epsn_z / e)

plt.ylim(ylim_m - ylim_d, ylim_m + ylim_d)

plt.xlabel('Turns')
plt.ylabel('$4\pi\epsilon_z$ [eV.s]');

$\implies$ this result shows that the nonlinearly matched thermal distribution is in equilibrium from the start (up to macro-particle noise, the fluctuations reduce with $1/\sqrt{N}$)!

<h2>Physical emittance growth: Dipole injection mismatch</h2>

In [ ]:
m = Machine(phi_s=0)

sigma_z = 8

z_ini, deltap_ini = generate_gaussian_in_rfbucket(N, sigma_z, m, margin=0.15)

Simulate a dipole injection mismatch (e.g. when the rf phase is not well synchronised between the injector and the synchrotron):

In [ ]:
z_ini -= 0.5 * sigma_z

4 meter mismatch in $z$ correspond to a phase mismatch of 16 degree:

In [ ]:
4 / (m.circumference / m.harmonic) * 360

In [ ]:
plot_mp(z_ini, deltap_ini / m.p0(), rfb=get_pyht_rfbucket(m), n_bins=20);

$\implies$ note the offset towards negative $z$, the contours of the macro-particle density are no longer matched to the Hamiltonian contours.

The safety `margin` inside the separatrix (where no particles are generated in `generate_gaussian_in_rfbucket`) should be chosen large enough such that no particles are located outside the rf bucket after the mismatch:

In [ ]:
assert all(hamiltonian(z_ini, deltap_ini, m) < 0), 'particles have been generated outside the rf bucket!'

Tracking the mismatched distribution of macro-particles:

In [ ]:
z = np.zeros((n_turns, N), dtype=np.float64)
deltap = np.zeros_like(z)

z[0] = z_ini
deltap[0] = deltap_ini

In [ ]:
for i_turn in trange(1, n_turns):
    z[i_turn], deltap[i_turn] = track_one_turn(z[i_turn - 1], deltap[i_turn - 1], m)

<h3>Centroid results</h3>

In [ ]:
plt.plot(np.mean(z, axis=1))

plt.xlabel('Turns')
plt.ylabel(r'$\langle z \rangle$ [m]');

$\implies$ exponential decay of the initial offset (due to the non-linearity of the rf bucket)

<h3>RMS bunch length results</h3>

In [ ]:
plt.plot(np.std(z, axis=1))

plt.xlabel('Turns')
plt.ylabel(r'$\sigma_z$ [m]');

$\implies$ saturation of the rms bunch length growth

<h3>RMS emittance results</h3>

In [ ]:
epsn_z = np.array([emittance(z_i, deltap_i) for z_i, deltap_i in zip(z, deltap)])

In [ ]:
plt.plot(epsn_z / e)

plt.xlabel('Turns')
plt.ylabel('$\epsilon_z$ [eV.s]');

$\implies$ in this example, 10% emittance growth as a result of the 4 meter injection offset.

In [ ]:
plot_mp(z[-1], deltap[-1] / m.p0(), rfb=get_pyht_rfbucket(m), n_bins=40);

$\implies$ the filamentation of the macro-particle distribution is clearly visible!

<h2>Physical emittance growth: Quadrupole injection mismatch</h2>

In [ ]:
m = Machine(phi_s=0)

sigma_z = 8

z_ini, deltap_ini = generate_gaussian_in_rfbucket(N, sigma_z, m, margin=0.15)

Simulate a quadrupole injection mismatch (e.g. when the rf voltage (rf bucket height) is not matched between the injector and the synchrotron):

In [ ]:
deltap_ini *= 0.5

In [ ]:
plot_mp(z_ini, deltap_ini / m.p0(), rfb=get_pyht_rfbucket(m), n_bins=20);

 $\implies$ note the squeezed rms momentum spread, the contours of the macro-particle density are no longer matched to the Hamiltonian contours.

Tracking the mismatched distribution of macro-particles:

In [ ]:
z = np.zeros((n_turns, N), dtype=np.float64)
deltap = np.zeros_like(z)

z[0] = z_ini
deltap[0] = deltap_ini

In [ ]:
for i_turn in trange(1, n_turns):
    z[i_turn], deltap[i_turn] = track_one_turn(z[i_turn - 1], deltap[i_turn - 1], m)

<h3>Centroid results</h3>

In [ ]:
plt.plot(np.mean(z, axis=1))

plt.xlabel('Turns')
plt.ylabel(r'$\langle z \rangle$ [m]');

$\implies$ only residual centroid fluctuations (due to macro-particle noise), note the amplitude of the oscillation in comparison to the rf bucket length!

<h3>RMS bunch length results</h3>

In [ ]:
plt.plot(np.std(z, axis=1))

plt.xlabel('Turns')
plt.ylabel(r'$\sigma_z$ [m]');

$\implies$ exponential decay of the initial momentum mismatch (due to the non-linearity of the rf bucket)

<h3>RMS emittance results</h3>

In [ ]:
epsn_z = np.array([emittance(z_i, deltap_i) for z_i, deltap_i in zip(z, deltap)])

In [ ]:
plt.plot(epsn_z / e)

plt.xlabel('Turns')
plt.ylabel('$\epsilon_z$ [eV.s]');

$\implies$ in this example, 20% emittance growth as a result of the 50% momentum spread mismatch.

In [ ]:
plot_mp(z[-1], deltap[-1] / m.p0(), rfb=get_pyht_rfbucket(m), n_bins=40);

$\implies$ again, the filamentation of the macro-particle distribution is clearly visible!